Ibu dan bapak.. code yang saya upload outputnya dihapus agar dapat diupload di Github. berikut versi link dengan output:

https://colab.research.google.com/drive/1TSiB6sJF6hi-aLhIKHU1skn_vGjqjAri?usp=sharing

disclaimer juga..  karena saya run berulang kembali menggunakan gpu, jadi saya run code ini di akun lain agar tidak perlu menggunakan colab pro.. izin bapak dan ibu 🙏🏻


In [ ]:
# Install PyTorch, Torchvision, and YOLOv5 dependencies
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118  # CUDA 11.8
!pip install opencv-python-headless  # For video processing
!pip install ultralytics  # YOLOv5 by Ultralytics

# Enable GPU acceleration in Colab under 'Runtime' > 'Change runtime type' > 'Hardware accelerator' > 'GPU'


In [ ]:
!gdown 1VYkFliBkwE53zIvsRU0P3A6LpIe-zJ-C

In [ ]:
data = ('/content/streetwalk.mov')

In [ ]:
from ultralytics import YOLO

# Load the pre-trained YOLOv5 model (e.g., YOLOv5s)
model = YOLO('yolov5s.pt')  # 's' stands for small model, optimized for speed

In [ ]:
import cv2

# Load video
video_path = '/content/streetwalk.mov'  # Update with your video path
cap = cv2.VideoCapture(video_path)

# Process frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO object detection on the frame
    results = model(frame)

    # Extract detection details
    for detection in results:
        boxes = detection.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Bounding box coordinates
            label = box.cls  # Class label
            confidence = box.conf  # Confidence score
            print(f"Detected {label} with confidence {confidence}")

cap.release()


In [ ]:
import cv2
from ultralytics import YOLO

# Load the pre-trained YOLOv5 model (e.g., YOLOv5s)
model = YOLO('yolov5su.pt')  # 's' stands for small model, optimized for speed

detections = []  # To store processed detection results

# Example preprocessing function
def preprocess_detections(results):
    for detection in results:
        boxes = detection.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Convert to integers
            label = box.cls
            confidence = box.conf
            detections.append({
                "label": label,
                "confidence": confidence,
                "bbox": [x1, y1, x2, y2]
            })

# Load video
video_path = '/content/streetwalk.mov'  # Update with your video path
cap = cv2.VideoCapture(video_path)

# Process frames and apply preprocessing within the loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO object detection on the frame
    results = model(frame)

    # Apply preprocessing for each frame's detections
    preprocess_detections(results)

cap.release()

In [ ]:
# Simple example of tracking based on previous detections
previous_detections = {}

def track_objects(detections):
    tracked_objects = []
    for det in detections:
        # Logic to assign consistent IDs to each detected object
        # For simplicity, we assume previous_detections exists with same structure
        for prev in previous_detections:
            if some_tracking_condition(det, prev):
                tracked_objects.append(det)  # Update tracking logic as needed
    return tracked_objects


In [ ]:
import torch
import cv2
from ultralytics import YOLO
import time
from google.colab.patches import cv2_imshow  # Colab-specific display function

# Check if GPU is available and set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load the YOLOv5 model and move it to the specified device
model = YOLO("yolov5su.pt")  # Small YOLOv5 model for faster inference
model.to(device)

# Set model confidence and IoU thresholds for optimization
model.conf = 0.4  # Confidence threshold to filter weak detections
model.iou = 0.45  # Intersection over Union (IoU) threshold for non-max suppression

# Set video source
video_path = '/content/streetwalk.mov'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Reduce frame resolution to optimize speed
width, height = 640, 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# Define display size for visualization (smaller size)
display_width, display_height = 320, 240  # Adjust to smaller dimensions

# Process every nth frame to reduce workload
frame_skip = 2  # Adjust frame skip for balance between speed and accuracy
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Skip frames based on frame_skip value
    if frame_count % frame_skip == 0:
        start_time = time.time()

        # Perform inference on the frame
        results = model(frame, device=device)

        # Process results
        for detection in results:
          boxes = detection.boxes
          for box in boxes:
              x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
              label = box.cls.item()  # Get the class label as an integer using .item()
              confidence = box.conf.item()  # Get confidence score as a float using .item()

              # Draw bounding box and label on frame
              cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
              cv2.putText(frame, f"{int(label)}: {confidence:.2f}", (x1, y1 - 10),  # Format label as integer
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)


        # Calculate FPS and display it on the frame
        end_time = time.time()
        fps = 1 / (end_time - start_time)
        cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Resize frame for a smaller display
        display_frame = cv2.resize(frame, (display_width, display_height))

        # Display the frame with detections
        cv2_imshow(display_frame)

    frame_count += 1

cap.release()
cv2.destroyAllWindows()


In [ ]:
# Set video source
video_path = '/content/streetwalk.mov'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Get the video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize VideoWriter to save the output video with detections
output_path = '/content/output_with_detections2.mp4'  # Output video path
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 format
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Process every nth frame to reduce workload
frame_skip = 2  # Adjust frame skip for balance between speed and accuracy
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Skip frames based on frame_skip value
    if frame_count % frame_skip == 0:
        start_time = time.time()

        # Perform inference on the frame
        results = model(frame, device=device)

        # Process results
        for detection in results:
            boxes = detection.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
                label = box.cls.item()  # Get the class label as an integer using .item()
                confidence = box.conf.item()  # Get confidence score as a float using .item()

                # Draw bounding box and label on frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f"{int(label)}: {confidence:.2f}", (x1, y1 - 10),  # Format label as integer
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Calculate FPS and display it on the frame
        end_time = time.time()
        fps = 1 / (end_time - start_time)
        cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Write the processed frame to the output video
        out.write(frame)

    frame_count += 1

# Release video capture and writer resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed video saved at: {output_path}")

In [ ]:
output_path

In [ ]:
output_path = '/content/output_with_detections.mp4'  # Output video path
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 format
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

In [ ]:
out.write(frame)

In [ ]:
out.release()

In [ ]:
from google.colab import files
files.download(output_path)

Hasil yang ditampilkan dalam tangkapan layar sudah cukup baik dan sesuai dengan output yang diharapkan dari model YOLO. Berikut penjelasannya:

1. **Objek yang Terdeteksi**:
   - Model mendeteksi berbagai objek seperti "persons" (orang), "car" (mobil), "bus" (bis), "backpacks" (tas ransel), dan "handbag" (tas tangan), dengan bounding box yang ditampilkan di sekitar masing-masing objek. Ini menunjukkan bahwa model mampu mengidentifikasi dan mengklasifikasikan banyak objek di area yang ramai, yang merupakan fitur utama YOLO.

2. **Bounding Box**:
   - Bounding box ditampilkan dengan akurat di sekitar objek yang terdeteksi, membantu menentukan lokasi setiap objek dalam frame. Ini adalah elemen penting dalam tugas deteksi objek.

3. **Kecepatan Inference (Prediksi)**:
   - Setiap frame menampilkan rincian waktu pemrosesan:
     - **Waktu preprocess**: Waktu yang dibutuhkan untuk mempersiapkan frame untuk YOLO.
     - **Waktu inference**: Waktu yang dibutuhkan YOLO untuk mendeteksi objek di dalam frame.
     - **Waktu postprocess**: Waktu untuk mempersiapkan output agar dapat divisualisasikan.
   - Waktu pemrosesan keseluruhan (misalnya, ~11,4 ms per frame) juga ditampilkan, memberikan gambaran tentang performa real-time.

4. **Resolusi Gambar**:
   - Resolusi 384x640 cukup baik untuk keseimbangan antara kecepatan dan kualitas deteksi. Ukuran ini umumnya cocok untuk aplikasi real-time atau hampir real-time.

Secara keseluruhan, output ini menunjukkan bahwa model bekerja dengan baik dalam hal deteksi objek, kecepatan, dan akurasi. Jika waktu pemrosesan dan kualitas deteksi sudah memenuhi kebutuhan proyek Anda, maka pengaturan ini sudah mencukupi. Namun, Anda masih bisa bereksperimen dengan optimisasi model jika memerlukan FPS yang lebih tinggi atau jika ada persyaratan kinerja tertentu."

# VIDEO

In [ ]:
!gdown 16CVtEieGWnpFy1yQtCcgJaVWTWIZdWM5

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load the YOLOv5 model and move it to the specified device
model = YOLO("yolov5su.pt")  # Small YOLOv5 model for faster inference
model.to(device)

# Set model confidence and IoU thresholds for optimization
model.conf = 0.4  # Confidence threshold to filter weak detections
model.iou = 0.45  # Intersection over Union (IoU) threshold for non-max suppression

# Set video source
video_path2 = '/content/5743518-hd_1920_1080_30fps.mp4'  # Replace with your video file path
cap2 = cv2.VideoCapture(video_path2)

# Reduce frame resolution to optimize speed
width, height = 640, 480
cap2.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap2.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# Define display size for visualization (smaller size)
display_width, display_height = 320, 240  # Adjust to smaller dimensions

# Process every nth frame to reduce workload
frame_skip = 2  # Adjust frame skip for balance between speed and accuracy
frame_count = 0

while cap2.isOpened():
    ret, frame = cap2.read()
    if not ret:
        break

    # Skip frames based on frame_skip value
    if frame_count % frame_skip == 0:
        start_time = time.time()

        # Perform inference on the frame
        results = model(frame, device=device)

        # Process results
        for detection in results:
          boxes = detection.boxes
          for box in boxes:
              x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
              label = box.cls.item()  # Get the class label as an integer using .item()
              confidence = box.conf.item()  # Get confidence score as a float using .item()

              # Draw bounding box and label on frame
              cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
              cv2.putText(frame, f"{int(label)}: {confidence:.2f}", (x1, y1 - 10),  # Format label as integer
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)


        # Calculate FPS and display it on the frame
        end_time = time.time()
        fps = 1 / (end_time - start_time)
        cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Resize frame for a smaller display
        display_frame = cv2.resize(frame, (display_width, display_height))

        # Display the frame with detections
        cv2_imshow(display_frame)

    frame_count += 1

cap.release()
cv2.destroyAllWindows()

In [ ]:
# Set video source
video_path2 = '/content/5743518-hd_1920_1080_30fps.mp4'  # Replace with your video file path
cap2 = cv2.VideoCapture(video_path2)

# Get the video properties
fps = cap2.get(cv2.CAP_PROP_FPS)
frame_width = int(cap2.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap2.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize VideoWriter to save the output video with detections
output_path2 = '/content/output_3.mp4'  # Output video path
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 format
out = cv2.VideoWriter(output_path2, fourcc, fps, (frame_width, frame_height))

# Process every nth frame to reduce workload
frame_skip = 2  # Adjust frame skip for balance between speed and accuracy
frame_count = 0

while cap2.isOpened():
    ret, frame = cap2.read()
    if not ret:
        break

    # Skip frames based on frame_skip value
    if frame_count % frame_skip == 0:
        start_time = time.time()

        # Perform inference on the frame
        results = model(frame, device=device)

        # Process results
        for detection in results:
            boxes = detection.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
                label = box.cls.item()  # Get the class label as an integer using .item()
                confidence = box.conf.item()  # Get confidence score as a float using .item()

                # Draw bounding box and label on frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f"{int(label)}: {confidence:.2f}", (x1, y1 - 10),  # Format label as integer
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Calculate FPS and display it on the frame
        end_time = time.time()
        fps = 1 / (end_time - start_time)
        cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Write the processed frame to the output video
        out.write(frame)

    frame_count += 1

# Release video capture and writer resources
cap2.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed video saved at: {output_path2}")

In [ ]:
from google.colab import files
files.download(output_path2)

In [ ]:
import numpy as np
from torchvision.ops import nms
from tqdm.auto import tqdm

# Initialize variables
detection_results = []
frame_index = 0
conf_threshold = 0.25
iou_threshold = 0.6

while True:
    ret, frame = cap2.read()
    if not ret:
        break

    # Convert frame to RGB for YOLO
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # YOLO inference
    results = model(frame_rgb)
    bbox_data = results.xyxy[0].cpu().numpy()
    boxes, scores, labels = bbox_data[:, :4], bbox_data[:, 4], bbox_data[:, 5]

    # NMS to filter boxes
    boxes_tensor = torch.tensor(boxes)
    scores_tensor = torch.tensor(scores)
    indices = nms(boxes_tensor, scores_tensor, iou_threshold)
    filtered_boxes = bbox_data[indices]

    # Draw detections
    for bbox in filtered_boxes:
        x1, y1, x2, y2, confidence, cls = bbox
        if confidence >= conf_threshold:
            label = model.names[int(cls)]
            color = (0, 255, 0)
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            cv2.putText(frame, f'{label} {confidence:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

            # Save results
            height, width, _ = frame.shape
            detection_results.append({
                'frame': frame_index,
                'x1': x1 / width,
                'y1': y1 / height,
                'x2': x2 / width,
                'y2': y2 / height,
                'confidence': confidence,
                'class': label
            })

    frame_index += 1
    out.write(frame)

cap2.release()
out.release()


In [ ]:
import pandas as pd
df_detections = pd.DataFrame(detection_results)
df_detections.to_csv('object_detection_results.csv', index=False)